###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from random import random
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix
#time.sleep(3600)

### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
find_the_way("small",".csv")

['small\\uk-vpn.csv', 'small\\uk.csv', 'small\\us-vpn.csv', 'small\\us.csv']

In [7]:
renamer={}

In [8]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['Ether_src']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [9]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):

    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [10]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [11]:
def MLCV(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        fold=5
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):


            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            ##df = df.reset_index(drop=True)
            df=df.fillna(0)
            macs=df["Ether_src"]
            del df["Ether_src"]
            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape

            for train_index, test_index in kfold.split(X):
                #try:
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index] 
                m_train, m_test = macs[train_index], macs[test_index] 

            
            

                results_y=[]
                cv+=1
                results_y.append(y_test)
    
    
                #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time=(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time=(float((time.time()-second)) )
                if step==1:
                    altime=0
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                else:
                    predict,altime=merged(m_test,predict,step,mixed)
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                    
                    
                lines.append(line)
                df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
                if cm.empty:
                    cm =df_cm
                else:
                    cm = cm.add(df_cm, fill_value=0)
                
        class_based_results=class_based_results/(repetition*fold)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()

        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//(repetition*fold)
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



In [12]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=25
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()

            m_train=df["Ether_src"]
            del df["Ether_src"]
            
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()

            m_test=df["Ether_src"]
            del df["Ether_src"]
            
            df["Label"]=df["Label"].replace(renamer)
            #df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [13]:
feature=feature_list=feature=col= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"Ether_src",'Label']

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

# Cross-Validation

In [15]:

ml_list={#"DT":DecisionTreeClassifier()}

    "RF":RandomForestClassifier(bootstrap=False,criterion="entropy",max_depth=19,max_features=6,min_samples_split=2,n_estimators=134)}

In [16]:
file_list={"UKVPN":'small/uk-vpn.csv',
           "UK":'small/uk.csv',
           "USVPN":'small/us-vpn.csv',
           "US":'small/us.csv'}

### Normal

In [17]:
savehere="results/cv/1/"
folder(savehere)
step=1
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    0.907    0.897   0.924  0.897  0.907  0.875      0   74.826     1.427          0
 1  UKVPN        0     2  RF    0.909    0.899   0.926  0.899  0.908  0.877      0   73.152     1.33           0
 2  UKVPN        0     3  RF    0.909    0.897   0.925  0.897  0.907  0.876      0   70.546     1.32           0
 3  UKVPN        0     4  RF    0.908    0.897   0.925  0.897  0.907  0.875      0   71.955     1.418          0
 4  UKVPN        0     5  RF    0.907    0.897   0.922  0.897  0.906  0.874      0   74.166     1.4            0
 5  UKVPN        1     1  RF    0.907    0.9     0.925  0.9    0.909  0.873      0   70.705     1.289          0
 6  UKVPN        1     2  RF    0.908    0.892   0.921  0.892  0.903  0.875      0   72.16

### Aggregated

In [18]:
savehere="results/cv/13/"
folder(savehere)
step=13
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    0.929    0.972   0.992  0.972  0.976  0.903      0   72.174     1.303      0.848
 1  UKVPN        0     2  RF    0.929    0.974   0.993  0.974  0.978  0.904      0   72.633     1.341      0.827
 2  UKVPN        0     3  RF    0.927    0.973   0.993  0.973  0.977  0.901      0   73.612     1.351      0.832
 3  UKVPN        0     4  RF    0.928    0.968   0.991  0.968  0.974  0.901      0   73.562     1.305      0.833
 4  UKVPN        0     5  RF    0.932    0.973   0.993  0.973  0.978  0.907      0   69.917     1.262      0.792
 5  UKVPN        1     1  RF    0.93     0.973   0.993  0.973  0.977  0.904      0   71.413     1.29       0.8
 6  UKVPN        1     2  RF    0.93     0.975   0.994  0.975  0.979  0.904      0   69.94  

# Session vs Session

In [28]:
file_list={"UKVPN@UK":['small/uk-vpn.csv','small/uk.csv'],
           "UK@UKVPN":['small/uk.csv','small/uk-vpn.csv'],
  "USVPN@US":['small/us-vpn.csv','small/us.csv'],
           "US@USVPN":['small/us.csv','small/us-vpn.csv']}

In [29]:
### Normal

In [30]:
say=0
savehere="results/ss/1/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.878    0.792   0.841  0.792  0.802  0.835      0   84.924     8.022          0
 1  UKVPN@UK     1     2  RF    0.878    0.79    0.84   0.79   0.801  0.835      0   83.186     6.009          0
 2  UKVPN@UK     2     3  RF    0.878    0.791   0.839  0.791  0.802  0.834      0   84.69      6.201          0
 3  UKVPN@UK     3     4  RF    0.878    0.791   0.841  0.791  0.802  0.835      0   84.532     6.546          0
 4  UKVPN@UK     4     5  RF    0.878    0.791   0.84   0.791  0.802  0.835      0   85.071     5.913          0
 5  UKVPN@UK     5     6  RF    0.878    0.792   0.841  0.792  0.803  0.835      0   83.506     6.008          0
 6  UKVPN@UK     6     7  RF    0.878    0.792   0.841  0.792  0.803  0.835      0   8

### Aggregated

In [31]:
say=0
savehere="results/ss/13/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.898    0.892   0.954  0.892  0.902  0.861      0   84.793     7.12       4.156
 1  UKVPN@UK     1     2  RF    0.899    0.893   0.956  0.893  0.905  0.861      0   84.793     5.994      4.535
 2  UKVPN@UK     2     3  RF    0.898    0.891   0.955  0.891  0.902  0.861      0   84.917     7.528      4.062
 3  UKVPN@UK     3     4  RF    0.899    0.897   0.954  0.897  0.907  0.861      0   83.605     5.973      4.074
 4  UKVPN@UK     4     5  RF    0.895    0.889   0.944  0.889  0.896  0.856      0   84.079     5.872      4.095
 5  UKVPN@UK     5     6  RF    0.896    0.89    0.949  0.89   0.9    0.858      0   83.593     5.905      4.069
 6  UKVPN@UK     6     7  RF    0.896    0.885   0.943  0.885  0.894  0.857      0   8

# Dataset Vs Dataset

In [17]:
file_list={"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
"UK@US":['small/uk.csv','small/us.csv'],
"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

### Normal

In [33]:
say=0
savehere="results/dd/1/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.741    0.725   0.742  0.725  0.719  0.673      0   85.88      6.021          0
 1  UKVPN@USVPN    1     2  RF    0.739    0.717   0.725  0.717  0.698  0.67       0   85.44      5.791          0
 2  UKVPN@USVPN    2     3  RF    0.74     0.725   0.738  0.725  0.716  0.672      0   81.066     5.906          0
 3  UKVPN@USVPN    3     4  RF    0.741    0.722   0.735  0.722  0.712  0.673      0   80.505     6.18           0
 4  UKVPN@USVPN    4     5  RF    0.74     0.724   0.742  0.724  0.718  0.672      0   83.248     5.899          0
 5  UKVPN@USVPN    5     6  RF    0.74     0.722   0.73   0.722  0.705  0.672      0   85.611     5.576          0
 6  UKVPN@USVPN    6     7  RF    0.738    0.717   0.725  0.717  0.

### Aggregated

In [ ]:
say=0
savehere="results/dd/13/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.836    0.843   0.902  0.843  0.861  0.784      0   80.987     5.561      3.607
 1  UKVPN@USVPN    1     2  RF    0.837    0.848   0.903  0.848  0.864  0.786      0   80.95      5.939      3.502
 2  UKVPN@USVPN    2     3  RF    0.836    0.844   0.902  0.844  0.861  0.785      0   85.655     6.593      3.346
 3  UKVPN@USVPN    3     4  RF    0.835    0.843   0.901  0.843  0.86   0.783      0   77.916     5.646      3.81
 4  UKVPN@USVPN    4     5  RF    0.837    0.845   0.901  0.845  0.862  0.786      0   83.975     5.696      3.63
 5  UKVPN@USVPN    5     6  RF    0.837    0.843   0.902  0.843  0.86   0.786      0   79.988     5.471      3.437
 6  UKVPN@USVPN    6     7  RF    0.837    0.847   0.902  0.847  0.86

In [18]:
file_list={#"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
#"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
#"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
#"UK@US":['small/uk.csv','small/us.csv'],
"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

In [19]:
say=0
savehere="results/dd/13/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


USVPN@UKVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  USVPN@UKVPN    0     1  RF    0.851    0.88    0.841  0.88   0.848  0.808      0   81.731    10.755      3.507
 1  USVPN@UKVPN    1     2  RF    0.85     0.877   0.841  0.877  0.846  0.807      0   77.618    10.415      3.379
 2  USVPN@UKVPN    2     3  RF    0.85     0.88    0.882  0.88   0.849  0.808      0   79.527    10.794      4.646
 3  USVPN@UKVPN    3     4  RF    0.849    0.88    0.882  0.88   0.849  0.807      0   85.699    10.735      3.928
 4  USVPN@UKVPN    4     5  RF    0.848    0.876   0.879  0.876  0.844  0.806      0   79.227    10.098      3.367
 5  USVPN@UKVPN    5     6  RF    0.849    0.877   0.881  0.877  0.847  0.807      0   79.096    10.934      3.554
 6  USVPN@UKVPN    6     7  RF    0.85     0.88    0.881  0.88   0.

In [ ]:
!shutdown /s /t 360